In [1]:
import pybamm as pb;import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;#import imageio
from scipy.io import savemat,loadmat;from pybamm import constants,exp;import matplotlib as mpl; fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
for k in range(0,1):
    mpl.rcParams["axes.labelsize"] = fs
    mpl.rcParams["axes.titlesize"] = fs
    mpl.rcParams["xtick.labelsize"] =  fs
    mpl.rcParams["ytick.labelsize"] =  fs
    mpl.rcParams["legend.fontsize"] =  fs
    mpl.rcParams['font.sans-serif'] = ['Times New Roman']
    mpl.rcParams['font.serif'] = ['Times New Roman']
    mpl.rcParams['axes.unicode_minus'] = False
import openpyxl
import traceback
import multiprocessing

In [2]:
V_max = 4.2; V_min = 2.5; 
exp_RPT_text = [ (f"Discharge at 1C until {V_min} V",  
        "Rest for 1 hours (5 minute period)",  
        f"Charge at 1C until {V_max} V (5 minute period)" ) ]
Experiment_RPT = pb.Experiment( exp_RPT_text * 1 ) 

Chemistry=pb.parameter_sets.OKane2022 

Para_0=pb.ParameterValues(chemistry=Chemistry) 

In [3]:
def neg_porosity(x_n):
   return 0.2 * (1-x_n/Para_0["Negative electrode thickness [m]"]) +0.05
Para_0.update({'Negative electrode porosity': neg_porosity})
Para_0.update({'Lithium plating kinetic rate constant [m.s-1]': 1e-9})

In [5]:
model_options = {"lithium plating":"partially reversible"}
Model_0 = pb.lithium_ion.DFN(options={
        "lithium plating": "partially reversible"
} ) #
Sim_0    = pb.Simulation(
    Model_0,        experiment = Experiment_RPT,
    parameter_values = Para_0,
    solver = pb.CasadiSolver(),) #mode="safe"
Sol_0    = Sim_0.solve(calc_esoh=False)

The linesearch algorithm failed with too small a step.
The linesearch algorithm failed with too small a step.


In [17]:
Para_0['Negative electrode porosity'] = "[input]"

#inputs={"Current function [A]": 1.6}

Sim_1   = pb.Simulation(
    Model_0,        experiment = Experiment_RPT,
    parameter_values = Para_0,
    solver = pb.CasadiSolver(),) #mode="safe"
Sol_0p05    = Sim_1.solve(
    inputs={'Negative electrode porosity':0.05})
Sol_0p1    = Sim_1.solve(
    inputs={'Negative electrode porosity':0.1})
Sol_0p15    = Sim_1.solve(
    inputs={'Negative electrode porosity':0.15})

In [21]:
Sol_0p06    = Sim_1.solve(inputs={'Negative electrode porosity':0.06})
Sol_0p07    = Sim_1.solve(inputs={'Negative electrode porosity':0.07})
Sol_0p08    = Sim_1.solve(inputs={'Negative electrode porosity':0.08})
Sol_0p09    = Sim_1.solve(inputs={'Negative electrode porosity':0.09})

In [10]:
Model_0.variables.search("potential")

Electrolyte potential
Electrolyte potential [V]
Gradient of electrolyte potential
Gradient of negative electrode potential
Gradient of negative electrolyte potential
Gradient of positive electrode potential
Gradient of positive electrolyte potential
Gradient of separator electrolyte potential
Negative current collector potential
Negative current collector potential [V]
Negative electrode lithium plating reaction overpotential
Negative electrode lithium plating reaction overpotential [V]
Negative electrode open circuit potential
Negative electrode open circuit potential [V]
Negative electrode potential
Negative electrode potential [V]
Negative electrode reaction overpotential
Negative electrode reaction overpotential [V]
Negative electrode surface potential difference
Negative electrode surface potential difference [V]
Negative electrolyte potential
Negative electrolyte potential [V]
Positive current collector potential
Positive current collector potential [V]
Positive electrode open ci

In [24]:
label = ["Linear","0.05","0.06","0.07","0.08","0.09","0.10","0.15"] 
output_variables3 = [
    "Porosity",
    "Terminal voltage [V]", 
    "Negative electrode lithium plating reaction overpotential [V]",
    "Loss of capacity to lithium plating [A.h]",
    "Lithium plating interfacial current density [A.m-2]",
]
quick_plot = pb.QuickPlot(
    [Sol_0,Sol_0p05,Sol_0p06,Sol_0p07,Sol_0p08,Sol_0p09,Sol_0p1,Sol_0p15], output_variables3,label,variable_limits='fixed',
    time_unit='hours',n_rows=2) #
quick_plot.dynamic_plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=2.624852733304113, step=0.02624852733304113)…

In [27]:
label = ["Linear","0.05","0.06","0.07","0.08","0.09","0.10","0.15"] 
output_variables3 = [
    "Porosity",
    "Current [A]",
    "Terminal voltage [V]", 
    "Positive electrode potential [V]",
    "Negative electrode potential [V]",
    "Electrolyte potential [V]",
    "Negative electrode lithium plating reaction overpotential [V]",
    "Loss of capacity to lithium plating [A.h]",
    "Lithium plating interfacial current density [A.m-2]",
]
quick_plot = pb.QuickPlot(
    [Sol_0,Sol_0p05,Sol_0p06,Sol_0p07,Sol_0p08,Sol_0p09,Sol_0p1,Sol_0p15], output_variables3,label,variable_limits='fixed',
    time_unit='hours',n_rows=3,figsize = (28,12),) #
quick_plot.dynamic_plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=2.624852733304113, step=0.02624852733304113)…

In [30]:
label = ["Linear","0.05"] 
output_variables3 = [
    "Porosity",
    "Electrolyte concentration [mol.m-3]",
    "Terminal voltage [V]", 
    "Positive electrode potential [V]",
    "Negative electrode potential [V]",
    "Electrolyte potential [V]",
    "Negative electrode lithium plating reaction overpotential [V]",

]
quick_plot = pb.QuickPlot(
    [Sol_0,Sol_0p05,], output_variables3,label,variable_limits='fixed',
    time_unit='hours',n_rows=3,) #figsize = (28,12),
quick_plot.dynamic_plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=2.3893989739113106, step=0.02389398973911310…